In [ ]:
# create a dataframe
# clean the data

# TODO: email CA about missing data from columns and what to do
# what is expected to be completed before the CA meeting and Maker Day

# Step 1. Mounting the Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import re

# Step 2. Creating the Dataframes

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1O5hZGn3uPlf0-HX8TjB3uKwttDP8E0YcnWXcTVR9QU0/edit?usp=sharing"
# Construct the URL for exporting as Excel
excel_url = sheet_url.replace('/edit?usp=sharing', '/export?format=xlsx')

all_sheets = pd.read_excel(excel_url, sheet_name=None)

# Create a list of the sheet names
sheet_names = list(all_sheets.keys())

# Display the sheet names
print("Sheet names:")
print(sheet_names)

Sheet names:
['Start here!', 'Public companies_intake form', 'Public companies_datasources', 'Full Data', 'Private companies_intake form', 'Private companies data categori', '1. Company Identity & Overview', '2. Location & Contact', '3. Company Type, Structure & In', '4. Size, Employees & People', '5. Online Presence & Media', '6. Investment Focus & Activity', '7. Performance & Financials', '8. News, Media & Updates']


In [ ]:
len(sheet_names)

14

In [ ]:
# df_dataset
# df_private_held.copy
# df_private_held_cleaned.copy

In [ ]:
for sheet_name, df in all_sheets.items():
    # Clean the sheet names
    cleaned_name = re.sub(r'\d+|!', '', sheet_name)  # remove digits and !
    cleaned_name = cleaned_name.replace(" ", "").replace("&", "_").replace("-", "_").replace(",", "_").replace(".", "").lower()

    # Create the DataFrame variable
    globals()[f'df_{cleaned_name}'] = df
    print(f"Created DataFrame: df_{cleaned_name}")

# df_full_data.head()
# display(df_full_data.head())

Created DataFrame: df_starthere
Created DataFrame: df_publiccompanies_intakeform
Created DataFrame: df_publiccompanies_datasources
Created DataFrame: df_fulldata
Created DataFrame: df_privatecompanies_intakeform
Created DataFrame: df_privatecompaniesdatacategori
Created DataFrame: df_companyidentity_overview
Created DataFrame: df_location_contact
Created DataFrame: df_companytype_structure_in
Created DataFrame: df_size_employees_people
Created DataFrame: df_onlinepresence_media
Created DataFrame: df_investmentfocus_activity
Created DataFrame: df_performance_financials
Created DataFrame: df_news_media_updates


# Q1. Company Legal Name (Jade)

In [ ]:
# data categories for legal name:
possible_name = ['name_lc', 'name_sl', 'companyName_ow', 'shortName_ow', 'name_cb', 'legal_name_cb', 'name_in', 'name_zi', 'company_gd']
df_company_name = df_companyidentity_overview[possible_name] # our new df to be modified from google sheet
print(df_company_name.isna().sum())
df_company_name.head(5)
print(df_company_name.shape)

name_lc             0
name_sl            70
companyName_ow    127
shortName_ow      127
name_cb            65
legal_name_cb     143
name_in           266
name_zi           144
company_gd        194
dtype: int64
(299, 9)


In [ ]:
def are_strings_same_in_row(row, columns_to_compare):
    # checks if the values in the row are the same and skips if the value is NaN
    values = [str(row[col]).lower() for col in columns_to_compare if pd.notna(row[col])]
    return all(v == values[0] for v in values)

# checks all the names
columns_to_compare = possible_name
df_company_name['are_names_same'] = df_company_name.apply(lambda row: are_strings_same_in_row(row, columns_to_compare), axis=1)

display(df_company_name.head())
print(df_company_name.shape)

# Create the new column 'legal_name', where is the names arent the same and the legal name isnt available, we default to the linkedin name
df_company_name['legal_name'] = np.where(
    (df_company_name['are_names_same'] == False) & (df_company_name['legal_name_cb'].isna()),
    df_company_name['name_lc'],
    np.where(df_company_name['are_names_same'] == True, df_company_name['name_lc'], df_company_name['legal_name_cb'])
)
# we default to the linkedin name because it is available for all companies, legal_name_cb might provide more context to what kind of company it is (we can default to linkedin name if its simpler)

# df with only the legal name
df_legal_name = df_company_name[['legal_name']]

display(df_legal_name.head())
print(df_legal_name.shape)

/tmp/ipython-input-2907121493.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_company_name['are_names_same'] = df_company_name.apply(lambda row: are_strings_same_in_row(row, columns_to_compare), axis=1)


,name_lc,name_sl,companyName_ow,shortName_ow,name_cb,legal_name_cb,name_in,name_zi,company_gd,are_names_same
0,Buckhead Wealth Management,Buckhead Wealth Management,Buckhead Wealth Management,Buckhead Wealth Management,Buckhead Wealth Management,NaN,NaN,NaN,Buckhead Wealth Management,True
1,Texas HALO Fund,Texas HALO Fund,The Texas HALO Fund,The Texas HALO Fund,Texas HALO Fund,"Texas HALO Fund, LLC",NaN,Texas Halo Fund,NaN,False
2,Shubh Investment Partners,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,Minutemen Alternative Investment Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,Liwa Capital Advisors,Liwa Capital Advisors,NaN,NaN,Liwa Capital Advisors,Liwa Capital Advisors LLC,NaN,NaN,NaN,False


(299, 10)


/tmp/ipython-input-2907121493.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_company_name['legal_name'] = np.where(


,legal_name
0,Buckhead Wealth Management
1,"Texas HALO Fund, LLC"
2,Shubh Investment Partners
3,Minutemen Alternative Investment Fund
4,Liwa Capital Advisors LLC


(299, 1)


Given that all companies have their linkedin names in the dataset, we can use it as the default feature for our model. To provide more context to the AI model, I've created code to take in 'legal_name_cb', in case we believe that could provide more context to the model and to our clients.

# Q9: Revenue Status & Growth (Jade)

In [ ]:
# """
# Current Revenue: ○ Pre-revenue ○ <$100K ARR ○ $100K-$1M ARR ○ $1M-$10M ARR ○ $10M-$50M ARR ○ >$50M ARR
# Growth Rate: ○ Pre-revenue ○ <50% ○ 50-100% ○ 100-200% ○ 200-500% ○ >500%


# # 7. performance & financials - revenue
# #         valuation & ipo
# #         traffic & activity metrics
# """

# missing revenue, how can we alternatively label these companies?
revenue_col = ['revenue_ow', 'revenueRange_ow', 'formattedRevenue_ow', 'details_revenue_gd', 'revenue_in', 'revenue_zi', 'revenue_currency_zi']

df_performance_financials.head(5)

df_revenue = pd.concat([df_legal_name, df_performance_financials[revenue_col]], axis=1)

print(df_revenue.isna().sum())
df_revenue.head(5)


legal_name               0
revenue_ow             127
revenueRange_ow        197
formattedRevenue_ow    127
details_revenue_gd     194
revenue_in             278
revenue_zi             144
revenue_currency_zi    144
dtype: int64


,legal_name,revenue_ow,revenueRange_ow,formattedRevenue_ow,details_revenue_gd,revenue_in,revenue_zi,revenue_currency_zi
0,Buckhead Wealth Management,0.0,NaN,< 1M,Unknown / Non-Applicable,NaN,NaN,NaN
1,"Texas HALO Fund, LLC",18000000.0,$5M - 25M,18M,NaN,NaN,5000000.0,USD
2,Shubh Investment Partners,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Minutemen Alternative Investment Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Liwa Capital Advisors LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Q11: Business Model & Market (Jade)

In [ ]:
"""
Model: ○ SaaS/Subscription ○ Marketplace ○ Transaction-based ○ E-commerce ○ Hardware ○ Freemium ○ Enterprise License ○ Advertising
Target Market: ○ SMB ○ Mid-Market ○ Enterprise ○ Consumers ○ Government ○ Mixed

"""

'\nModel: ○ SaaS/Subscription ○ Marketplace ○ Transaction-based ○ E-commerce ○ Hardware ○ Freemium ○ Enterprise License ○ Advertising\nTarget Market: ○ SMB ○ Mid-Market ○ Enterprise ○ Consumers ○ Government ○ Mixed\n\n'

In [ ]:
df_companytype_structure_in.isna().sum()

,0
organization_type_lc,0
type_sl,92
ownership_ow,127
industrySectors_ow,186
industries_lc,0
industries_sl,70
industries_ow,140
industries_cb,81
industry_in,288
industry_zi,144


# Q14: Funding History & Capital Efficiency (Jade)

In [ ]:
# monthly burn & runway
"""
Total Funding Raised: ○ $0 ○ <$500K ○ $500K-$2M ○ $2M-$10M ○ $10M-$50M ○ >$50M
Notable Previous Investors: _________________
Current Monthly Burn: ○ <$25K ○ $25K-$100K ○ $100K-$500K ○ $500K-$1M ○ >$1M
Current Runway: ○ <6 months ○ 6-12 months ○ 12-18 months ○ 18+ months
"""

'\nTotal Funding Raised: ○ $0 ○ <$500K ○ $500K-$2M ○ $2M-$10M ○ $10M-$50M ○ >$50M\nNotable Previous Investors: _________________\nCurrent Monthly Burn: ○ <$25K ○ $25K-$100K ○ $100K-$500K ○ $500K-$1M ○ >$1M\nCurrent Runway: ○ <6 months ○ 6-12 months ○ 12-18 months ○ 18+ months\n'

In [ ]:
df_investmentfocus_activity[0:300].isna().sum()

# df_investmentfocus_activity.shape # fix shape to

,0
name_lc,1
investor_type_cb,287
investment_stage_cb,291
investment_stage_zi,300
investment_preferences,300
funding_lc,233
investors_lc,274
totalFunding_ow,128
totalFundings_ow,128
totalInvestments_ow,128


# Q19: Long-term Vision & Exit Path (Jade)

In [ ]:
'''
5-Year Vision: ○ Category leader ○ Strategic acquisition target ○ IPO candidate ○ Profitable growth ○ International expansion ○ Platform/ecosystem
Expected Exit Timeline: ○ 3-5 years ○ 5-7 years ○ 7-10 years ○ Building for long-term ○ Too early to determine
Likely Exit Path: ○ Strategic acquisition ○ IPO ○ Management buyout ○ Too early to determine
'''

'\n5-Year Vision: ○ Category leader ○ Strategic acquisition target ○ IPO candidate ○ Profitable growth ○ International expansion ○ Platform/ecosystem\nExpected Exit Timeline: ○ 3-5 years ○ 5-7 years ○ 7-10 years ○ Building for long-term ○ Too early to determine\nLikely Exit Path: ○ Strategic acquisition ○ IPO ○ Management buyout ○ Too early to determine\n'